# Assessment Planning Tool

This notebook helps you generate an optimal schedule for assessment days, given constraints on candidates, assessors, exercise durations, and required breaks.

- Each candidate completes 3 exercises: Roleplay (40 min), Case (40 min), ITV (60 min)
- Breaks of at least 20 minutes between exercises
- Each exercise requires a dedicated assessor
- Number of assessors can be set (minimum 3)
- Assessment day starts at 12:30

Fill in the parameters below and run the scheduling tool to generate your plan.

In [501]:
# Parameters for the assessment planning tool
from datetime import datetime, timedelta

# User input: adjust as needed
num_candidates = 5
# Specify the number of assessors per exercise type (e.g. {"Roleplay": 1, "Case": 1, "ITV": 2})
assessors_per_exercise = {"Roleplay": 1, "Case": 1, "ITV": 1}
start_time = "12:30"  # Format: HH:MM
candidate_break_minutes = 20
assessor_break_minutes = 10 

# Exercise definitions
exercises = [
    {"name": "Roleplay", "duration": 40},
    {"name": "Case", "duration": 40},
    {"name": "ITV", "duration": 60},
]

In [502]:
# Optional: Per-candidate exercise durations (in minutes)
# Uncomment and edit as needed. If not provided, defaults from 'exercises' will be used.
# Example:
exercise_durations = {
    "Candidate 1": {"Roleplay": 40, "Case": 40, "ITV": 40},
    "Candidate 2": {"Roleplay": 40, "Case": 40, "ITV": 40},
    "Candidate 3": {"Roleplay": 40, "Case": 40, "ITV": 40},
    "Candidate 4": {"Roleplay": 40, "Case": 40, "ITV": 60},
}
exercise_durations = None  # Set to None to use default durations


In [503]:
import pandas as pd
from IPython.display import display, HTML

def generate_compact_schedule_multiassessors(num_candidates, assessors_per_exercise, start_time, candidate_break_minutes, assessor_break_minutes, exercises, exercise_durations=None):
    # Build list of all assessor slots (e.g., ITV-1, ITV-2)
    assessor_slots = []
    for ex in exercises:
        for i in range(assessors_per_exercise.get(ex['name'], 1)):
            if assessors_per_exercise.get(ex['name'], 1) == 1:
                assessor_slots.append(ex['name'])
            else:
                assessor_slots.append(f"{ex['name']}-{i+1}")
    # For each candidate, keep track of which exercises are left
    candidate_ex_left = {f"Candidate {i+1}": [ex["name"] for ex in exercises] for i in range(num_candidates)}
    # Precompute total workload and max single exercise for each candidate
    def get_total_workload(cand):
        if exercise_durations and cand in exercise_durations:
            return sum(exercise_durations[cand].get(ex['name'], ex['duration']) for ex in exercises)
        else:
            return sum(ex['duration'] for ex in exercises)
    def get_max_ex_duration_left(cand):
        if exercise_durations and cand in exercise_durations:
            return max([exercise_durations[cand].get(ex, next(e['duration'] for e in exercises if e['name']==ex)) for ex in candidate_ex_left[cand]] or [0])
        else:
            return max([next(e['duration'] for e in exercises if e['name']==ex) for ex in candidate_ex_left[cand]] or [0])
    candidate_workload = {cand: get_total_workload(cand) for cand in candidate_ex_left}
    # Track when each candidate is next available (all start at 12:30)
    candidate_free = {cand: pd.Timestamp(start_time) for cand in candidate_ex_left}
    # Track when each assessor slot is next available (can start at 0:00, so not forced to start at 12:30)
    assessor_free = {slot: pd.Timestamp('1900-01-01 00:00') for slot in assessor_slots}
    # For each exercise type, keep a list of its assessor slots
    ex_to_slots = {}
    for ex in exercises:
        slots = []
        for i in range(assessors_per_exercise.get(ex['name'], 1)):
            if assessors_per_exercise.get(ex['name'], 1) == 1:
                slots.append(ex['name'])
            else:
                slots.append(f"{ex['name']}-{i+1}")
        ex_to_slots[ex['name']] = slots
    schedule = []
    # While there are still exercises to schedule
    while any(candidate_ex_left[cand] for cand in candidate_ex_left):
        # Find all possible next slots (candidate, exercise, assessor slot, time)
        possible = []
        for cand in candidate_ex_left:
            if not candidate_ex_left[cand]:
                continue
            for ex_name in candidate_ex_left[cand]:
                for slot in ex_to_slots[ex_name]:
                    t_cand = candidate_free[cand]
                    t_assessor = assessor_free[slot]
                    t = max(t_cand, t_assessor)
                    possible.append((cand, ex_name, slot, t))
        if not possible:
            break
        # Among all possible, pick those with the earliest time
        min_time = min(p[3] for p in possible)
        earliest = [p for p in possible if p[3] == min_time]
        # Among those, prioritize candidates who still have their longest exercise left (e.g. ITV)
        def is_longest_left(cand, ex_name):
            max_left = get_max_ex_duration_left(cand)
            if exercise_durations and cand in exercise_durations:
                dur = exercise_durations[cand].get(ex_name, next(e['duration'] for e in exercises if e['name']==ex_name))
            else:
                dur = next(e['duration'] for e in exercises if e['name']==ex_name)
            return dur == max_left
        # Sort: longest exercise left, then largest workload, then candidate number
        earliest.sort(key=lambda x: (not is_longest_left(x[0], x[1]), -candidate_workload[x[0]], x[0]))
        cand, ex_name, slot, st = earliest[0]
        # Determine duration: per-candidate if provided, else default
        if exercise_durations and cand in exercise_durations and ex_name in exercise_durations[cand]:
            duration = exercise_durations[cand][ex_name]
        else:
            duration = [ex for ex in exercises if ex['name']==ex_name][0]['duration']
        et = st + pd.Timedelta(minutes=duration)
        schedule.append({
            "Candidate": cand,
            "Exercise": ex_name,
            "Assessor": slot,
            "Start": st,
            "End": et
        })
        # Candidate must have a break after each activity (except last)
        if candidate_ex_left[cand] and len(candidate_ex_left[cand]) > 1:
            candidate_free[cand] = et + pd.Timedelta(minutes=candidate_break_minutes)
        else:
            candidate_free[cand] = et
        # Assessor must have a break after each activity (except last for that assessor)
        assessor_free[slot] = et + pd.Timedelta(minutes=assessor_break_minutes)
        candidate_ex_left[cand].remove(ex_name)
    # Shift all so the earliest start is 12:30 (if not already)
    min_start = min(s['Start'] for s in schedule)
    shift = pd.Timestamp(start_time) - min_start
    for s in schedule:
        s['Start'] += shift
        s['End'] += shift
    df = pd.DataFrame(schedule)
    # --- Remove shifting of last activities to avoid overlaps ---
    return df

df_schedule = generate_compact_schedule_multiassessors(num_candidates, assessors_per_exercise, start_time, candidate_break_minutes, assessor_break_minutes, exercises, exercise_durations=exercise_durations)

In [504]:
# Build and display the color-coded timetable with breaks labeled
def color_block(candidate):
    colors = {
        'Candidate 1': '#e6f7ff',
        'Candidate 2': '#fff7e6',
        'Candidate 3': '#e6ffe6',
        'Candidate 4': '#ffe6f7',
    }
    return f'background-color: {colors.get(candidate, "#f0f0f0")};' if candidate and candidate.startswith('Candidate') else ''

def build_timetable_multiassessors(df_schedule, exercises, assessors_per_exercise, start_time="12:30", interval=10):
    # Determine time range
    min_start = df_schedule['Start'].min()
    max_end = df_schedule['End'].max()
    time_slots = pd.date_range(min_start.floor('T'), max_end.ceil('T'), freq=f'{interval}min')
    # Build columns: one per assessor slot
    assessor_slots = []
    for ex in exercises:
        for i in range(assessors_per_exercise.get(ex['name'], 1)):
            if assessors_per_exercise.get(ex['name'], 1) == 1:
                assessor_slots.append(ex['name'])
            else:
                assessor_slots.append(f"{ex['name']}-{i+1}")
    timetable = pd.DataFrame(index=time_slots, columns=assessor_slots)
    # Fill timetable (INCLUSIVE of last slot)
    for _, row in df_schedule.iterrows():
        slot = row['Assessor']
        candidate = row['Candidate']
        exercise = row['Exercise']
        # Use End - 1 minute to include the last interval
        for t in pd.date_range(row['Start'], row['End'] - pd.Timedelta(minutes=1), freq=f'{interval}min'):
            timetable.at[t, slot] = f"{candidate} ({exercise})"
    # Fill empty slots as Break
    timetable = timetable.fillna('Break')
    # Style
    def style_func(val):
        if val == 'Break':
            return 'color: #888; background-color: #f0f0f0; font-style: italic;'
        elif '(' in val:
            candidate = val.split('(')[0].strip()
            return color_block(candidate)
        return ''
    styled_tt = timetable.style.applymap(style_func)
    display(HTML('<h3>Assessment Timetable</h3>'))
    display(styled_tt)

timetable = build_timetable_multiassessors(df_schedule, exercises, assessors_per_exercise)

/tmp/ipykernel_4281/864559487.py:15: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_slots = pd.date_range(min_start.floor('T'), max_end.ceil('T'), freq=f'{interval}min')
/tmp/ipykernel_4281/864559487.py:43: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_tt = timetable.style.applymap(style_func)


,Roleplay,Case,ITV
2025-09-20 12:30:00,Candidate 2 (Roleplay),Candidate 3 (Case),Candidate 1 (ITV)
2025-09-20 12:40:00,Candidate 2 (Roleplay),Candidate 3 (Case),Candidate 1 (ITV)
2025-09-20 12:50:00,Candidate 2 (Roleplay),Candidate 3 (Case),Candidate 1 (ITV)
2025-09-20 13:00:00,Candidate 2 (Roleplay),Candidate 3 (Case),Candidate 1 (ITV)
2025-09-20 13:10:00,Break,Break,Candidate 1 (ITV)
2025-09-20 13:20:00,Candidate 4 (Roleplay),Candidate 5 (Case),Candidate 1 (ITV)
2025-09-20 13:30:00,Candidate 4 (Roleplay),Candidate 5 (Case),Break
2025-09-20 13:40:00,Candidate 4 (Roleplay),Candidate 5 (Case),Candidate 2 (ITV)
2025-09-20 13:50:00,Candidate 4 (Roleplay),Candidate 5 (Case),Candidate 2 (ITV)
2025-09-20 14:00:00,Break,Break,Candidate 2 (ITV)
